In [2]:
from collections import namedtuple
import os
os.chdir(r"C:\Users\Praveen\deepCNNClassifier\deepCNNClassifier") # making the parent directory as jupyter notebook consuder the research folder
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augumentation: bool
    params_image_size: list


In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self)-> TrainingConfig:
        training = self.config.training  # all values that are defined configs.yaml availabe here
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")

        create_directories([training.artifacts_root])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augumentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )
        return training_config

In [7]:
# trainingComponent
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs= dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],# no need to pass channel
            batch_size= self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=false,
            **dataflow_kwargs
        )

        if self.config.params_is_augumentation:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2, # applying some type of pressure, so it will tilt the image
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            training_datagenerator=valid_datagenerator

        self.train_generator = training_datagenerator.flow_from_directory(
            self.config.training_data,
            subset="Training",
            shuffle=True,
            **dataflow_kwargs
        )

    def train(self):
        self.steps_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.train_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epochs=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model

        )
    

In [8]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_ckpt_callbacks()
    
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e

AttributeError: 'ConfigurationManager' object has no attribute 'get_prepare_callback_config'

In [9]:
def example(x, **kwargs):
    print(locals())

In [11]:
example(x=3, y=4, z=0)

{'x': 3, 'kwargs': {'y': 4, 'z': 0}}


In [12]:
# same defined it as like below
extra= dict(a=4,b=3)

example(x=3, **extra)

{'x': 3, 'kwargs': {'a': 4, 'b': 3}}
